In [1]:
import pandas as pd 
import plotly.express as px

data = pd.read_csv('videos_export.csv')
data['uploaded_at'] = pd.to_datetime(data['uploaded_at'])

In [2]:
totals = data.groupby(['uploaded_at']).count()
totals = pd.DataFrame({"Total_Counts" : totals.codec})
totals['Total_Aggregation'] = totals.Total_Counts.cumsum()
totals.head()

,Total_Counts,Total_Aggregation
uploaded_at,,
2018-12-05,2,2
2018-12-06,6,8
2018-12-07,2,10
2018-12-08,10,20
2018-12-09,14,34


In [3]:
plot_df = []

for codec in data['codec'].unique():
    temp = data[data.codec == codec]
    temp = temp.groupby(['uploaded_at']).count()

    temp = pd.DataFrame({"Date":temp.index, "Codec": codec, "Count" : temp.codec})
    temp = pd.concat([temp, totals], axis=1)
    temp['Count_Percentage'] = 100*temp.Count/temp.Total_Counts
    temp['Aggregation'] = temp.Count.cumsum()
    temp['Aggregation_Percentage'] = 100*temp.Aggregation/temp.Total_Aggregation

    plot_df.append(temp)

In [4]:
plot_df = pd.concat(plot_df)
plot_df = plot_df.sort_values(by=['Date'])
plot_df.head()

,Date,Codec,Count,Total_Counts,Total_Aggregation,Count_Percentage,Aggregation,Aggregation_Percentage
uploaded_at,,,,,,,,
2018-12-05,2018-12-05,h264,2.0,2,2,100.000000,2.0,100.0
2018-12-06,2018-12-06,vp8,2.0,6,8,33.333333,2.0,25.0
2018-12-06,2018-12-06,h264,4.0,6,8,66.666667,6.0,75.0
2018-12-07,2018-12-07,h264,2.0,2,10,100.000000,8.0,80.0
2018-12-08,2018-12-08,h264,8.0,10,20,80.000000,16.0,80.0


In [10]:
temp = plot_df.dropna(subset=['Date','Aggregation_Percentage'])

fig = px.line(temp, x = "Date", y = "Aggregation_Percentage", color='Codec')
fig.show()

In [ ]:
fig.write_image("counts.png")